In [12]:
import json
import os
import sys 
import tensorflow as tf
import pandas as pd

#Create and open a new .csv file to load all the features from the json files
featuredata=open(r"Documents/DATestFeatures.csv","w")
L = ["File Name,","Files Deleted,", "Files Created,","Files Written,"
     ,"Directories Created,","Opened REG keys,","DLLs Loaded,","Hosts,","VirusTotal\n"] 
featuredata.writelines(L) 

#Folder where the files whose value has to be predicted are stored
FILEPATH = '/home/ghildyaa/Documents/Projects 3 and 4/Dynamic'
entries = os.listdir (FILEPATH) 
filecount = 1

for entry in entries:
    #print(str(filecount)+": "+entry)
    filecount=filecount+1
   
    with open(FILEPATH +'/'+entry) as f:
      data = json.load(f)

    featuredata.write(entry+",") #File Name - This will be required later to load the Output file
    #featuredata.write(str(x-1) + ",") #File Type - This will not be available for the fileset to be predicted

    try:
        featuredata.write(str(len(data['behavior']['summary']['file_deleted']))+",")
    except:
        featuredata.write("0,")

    try:
        featuredata.write(str(len(data['behavior']['summary']['file_created']))+",")
    except:
        featuredata.write('0,')

    try:
        featuredata.write(str(len(data['behavior']['summary']['file_written']))+",")
    except:
        featuredata.write('0,')

    try:
        featuredata.write(str(len(data['behavior']['summary']['directory_created']))+",")
    except:
        featuredata.write('0,')

    try:
        featuredata.write(str(len(data['behavior']['summary']['regkey_opened']))+",")
    except:
        featuredata.write('0,')

    try:
        featuredata.write(str(len(data['behavior']['summary']['dll_loaded']))+",")
    except:
        featuredata.write('0,')

    try:
        featuredata.write(str(len(data['network']['hosts']))+",")
    except:
        featuredata.write('0,')

    try:
        featuredata.write(str(data['virustotal']['summary']['positives'])+"\n")
    except:
        featuredata.write('0\n')    

    
featuredata.close() 
#print("End")


#Load result file 
outputdata=open(r"Documents/DAMLOutput.csv","w") 
L = ["File_Name,","File_Type\n"] 
outputdata.writelines(L) 

#Load file to be predicted 
test_file = pd.read_csv('Documents/DATestFeatures.csv') 

file_name = (test_file['File Name']) #Load file name into a different array since we don't need it for prediction

Final_test = test_file.drop(columns=['File Name']) #Remove the file name from the dataset to be predicted

#Load saved Model file - saves to Home directory by default
model = tf.keras.models.load_model("MalDA.model")

#Pass the dataset to be predicted to the Model
pred_test= model.predict_classes(Final_test) 

#Iterage through the predicted values and add an entry in the Result file
i=0 
while i < len(pred_test): 
    if pred_test[i] == 6: 
        outputdata.write(file_name[i] + ",Benign\n")
    elif pred_test[i] == 5:
        outputdata.write(file_name[i] + ",Worm\n")
    elif pred_test[i] == 4:
        outputdata.write(file_name[i] + ",Virus\n")
    elif pred_test[i] == 3:
        outputdata.write(file_name[i] + ",TrojanDropper\n")
    elif pred_test[i] == 2:
        outputdata.write(file_name[i] + ",TrojanDownloader\n")
    elif pred_test[i] == 1:
        outputdata.write(file_name[i] + ",Trojan\n")
    elif pred_test[i] == 0:
        outputdata.write(file_name[i] + ",Backdoor\n")
    i=i+1

outputdata.close()

print ("Finished")


Finished
